In [15]:
# IMPORT PANDAS / DATA MANIPULATION MODULES
import pandas as pd
import numpy as np
import os
# import glob
import csv
import math
import datetime

In [16]:
# IMPORT CHARTING
import seaborn as sns
import matplotlib.pyplot as plt
import panel as pn

In [17]:
# INITIALIZE PANEL
# pn.extension(sizing_mode='stretch_width')
pn.extension('tabulator', sizing_mode='stretch_width')
import hvplot.pandas

In [18]:
# ADDING TO HIDE RED SQUARES
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [19]:
# IMPORT FOR FORMATTING
from bokeh.models.widgets.tables import NumberFormatter, BooleanFormatter, StringFormatter
# https://panel.holoviz.org/reference/widgets/Tabulator.html
# FORMATTING: https://panel.holoviz.org/reference/widgets/Tabulator.html#formatters
# https://tabulator.info/docs/4.9/format#format-builtin

# Data Import and Cleanup

In [20]:
# IMPORT DATA
# FILE PATH
path = './DATAOUTPUTS/df_all_gdp_ppi.csv'
# IMPORT
df_gdp_ppi_import = pd.read_csv(path)

In [21]:
df_gdp_ppi_import.head(2)

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,GDP,Flag Codes,Lookup Area,Lookup M49,...,Item Code (CPC),Item,Element Code,Element,Months Code,Months,Unit,YEAR,wheat_PPI,YEAR_INT
0,AUT,GDP,TOT,USD_CAP,A,2000,29380.031110,NaN,Austria,40.0,...,'0111,Wheat,5539,Producer Price Index (2014-2016 = 100),7021,Annual value,NaN,Y2000,85.69,2000
1,AUT,GDP,TOT,USD_CAP,A,2001,29707.462264,NaN,Austria,40.0,...,'0111,Wheat,5539,Producer Price Index (2014-2016 = 100),7021,Annual value,NaN,Y2001,82.98,2001


In [22]:
df_gdp_ppi_import.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   LOCATION         682 non-null    object 
 1   INDICATOR        682 non-null    object 
 2   SUBJECT          682 non-null    object 
 3   MEASURE          682 non-null    object 
 4   FREQUENCY        682 non-null    object 
 5   TIME             682 non-null    int64  
 6   GDP              682 non-null    float64
 7   Flag Codes       37 non-null     object 
 8   Lookup Area      682 non-null    object 
 9   Lookup M49       682 non-null    float64
 10  Lookup iso       682 non-null    object 
 11  Area Code        682 non-null    int64  
 12  Area Code (M49)  682 non-null    object 
 13  Area             682 non-null    object 
 14  eu_member        682 non-null    int64  
 15  Item Code        682 non-null    int64  
 16  Item Code (CPC)  682 non-null    object 
 17  Item            

In [23]:
df_gdp_ppi_import.columns

Index(['LOCATION', 'INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'TIME',
       'GDP', 'Flag Codes', 'Lookup Area', 'Lookup M49', 'Lookup iso',
       'Area Code', 'Area Code (M49)', 'Area', 'eu_member', 'Item Code',
       'Item Code (CPC)', 'Item', 'Element Code', 'Element', 'Months Code',
       'Months', 'Unit', 'YEAR', 'wheat_PPI', 'YEAR_INT'],
      dtype='object')

In [24]:
# REMOVE DUPLICATE FIELDS
df_gdp_ppi_removing_columns = df_gdp_ppi_import[['LOCATION', 'INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'TIME', 'GDP', 'Flag Codes', 'Lookup iso','Area Code', 'Area Code (M49)', 'Area', 'eu_member', 'Item Code', 'Item Code (CPC)', 'Item', 'Element Code', 'Element', 'Months Code', 'Months', 'Unit', 'wheat_PPI'                                          ]]
# 

In [25]:
# PRINT UGLY COLUMNS
df_gdp_ppi_removing_columns.columns
# 

Index(['LOCATION', 'INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'TIME',
       'GDP', 'Flag Codes', 'Lookup iso', 'Area Code', 'Area Code (M49)',
       'Area', 'eu_member', 'Item Code', 'Item Code (CPC)', 'Item',
       'Element Code', 'Element', 'Months Code', 'Months', 'Unit',
       'wheat_PPI'],
      dtype='object')

In [26]:
# RENAME COLUMNS
df_gdp_ppi_removing_columns.columns = df_gdp_ppi_removing_columns.columns.str.upper()
df_gdp_ppi_removing_columns.columns = df_gdp_ppi_removing_columns.columns.str.replace(" ", "_")
df_gdp_ppi_removing_columns.columns = df_gdp_ppi_removing_columns.columns.str.replace("(", "", regex=True)
df_gdp_ppi_removing_columns.columns = df_gdp_ppi_removing_columns.columns.str.replace(")", "", regex=True)
df_gdp_ppi_removing_columns.rename(columns = {'TIME':'YEAR'}, inplace = True)
df_gdp_ppi_removing_columns['EU_STRING'] = np.where(df_gdp_ppi_removing_columns['EU_MEMBER'] == 1, "EU", "NON-EU")
# df_gdp_ppi_removing_columns['YEAR'] = df_gdp_ppi_removing_columns['TIME']
df_gdp_ppi_removing_columns.columns

Index(['LOCATION', 'INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'YEAR',
       'GDP', 'FLAG_CODES', 'LOOKUP_ISO', 'AREA_CODE', 'AREA_CODE_M49', 'AREA',
       'EU_MEMBER', 'ITEM_CODE', 'ITEM_CODE_CPC', 'ITEM', 'ELEMENT_CODE',
       'ELEMENT', 'MONTHS_CODE', 'MONTHS', 'UNIT', 'WHEAT_PPI', 'EU_STRING'],
      dtype='object')

In [27]:
# df_gdp_ppi_removing_columns.head(2)

In [28]:
# FINAL DATAFRAME == df_gdp_ppi
df_gdp_ppi = df_gdp_ppi_removing_columns
# 
df_gdp_ppi.head(2)

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,YEAR,GDP,FLAG_CODES,LOOKUP_ISO,AREA_CODE,...,ITEM_CODE,ITEM_CODE_CPC,ITEM,ELEMENT_CODE,ELEMENT,MONTHS_CODE,MONTHS,UNIT,WHEAT_PPI,EU_STRING
0,AUT,GDP,TOT,USD_CAP,A,2000,29380.031110,NaN,AUT,11,...,15,'0111,Wheat,5539,Producer Price Index (2014-2016 = 100),7021,Annual value,NaN,85.69,EU
1,AUT,GDP,TOT,USD_CAP,A,2001,29707.462264,NaN,AUT,11,...,15,'0111,Wheat,5539,Producer Price Index (2014-2016 = 100),7021,Annual value,NaN,82.98,EU


# CREATING VIS PANELS

Modern farming has a great dependence on technology and relies upon visualizations to communicate information, this includes web based, mobile based and many other digital transmission formats. Develop an interactive dashboard tailored to modern farmers, using tufts principles, to showcase the information/evidence gathered following your Machine Learning Analysis. Detail the rationale for approach and visualisation choices made during development. Note you may not use Powerbi, rapidminer, tableau or other such tools to accomplish this (at this stage)

In [29]:
# VIS TO CREATE
# 
# BREAKDOWN BY COUNTRY
# 

In [30]:
# GET COUNTRY (AREA) LIST
# df_gdp_ppi.AREA.unique().tolist()
# DEFAULT COUNTRY
default_country = "Ireland"
# COUNTRY SELECTOR FROM DATA
country_selector = pn.widgets.Select(value=default_country, options=df_gdp_ppi.AREA.unique().tolist(),)
# DISPLAY TEMPORARILY
pn.Row(pn.Column('#### Select Country', country_selector ), pn.Column())

Row(sizing_mode='stretch_width')
    [0] Column(sizing_mode='stretch_width')
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] Select(options=['Austria', 'Belgium', ...], sizing_mode='stretch_width', value='Ireland')
    [1] Column(sizing_mode='stretch_width')

In [31]:
# SELECTED COLUMNS
# print(df_gdp_ppi.columns)
# REARRANGE COLUMNS
# ['LOCATION', 
# 'INDICATOR', 
# 'SUBJECT', 
# 'MEASURE', 
# 'FREQUENCY',
# 'YEAR',
# 'GDP',
# 'FLAG_CODES',
# 'LOOKUP_ISO',
# 'AREA_CODE',
# 'AREA_CODE_M49',
# 'AREA',
# 'EU_MEMBER',
# 'ITEM_CODE',
# 'ITEM_CODE_CPC', 
# 'ITEM', 
# 'ELEMENT_CODE',
# 'ELEMENT', 
# 'MONTHS_CODE', 
# 'MONTHS', 
# 'UNIT', 
# 'WHEAT_PPI', 
# 'EU_STRING']
df_gdp_ppi_reordered = df_gdp_ppi[[
    'YEAR',
    'LOCATION', 
    'FLAG_CODES',
    'LOOKUP_ISO',
    'AREA_CODE',
    'AREA_CODE_M49',
    'AREA',
    'INDICATOR', 
    'SUBJECT', 
    'MEASURE', 
    'FREQUENCY',
    'EU_MEMBER',
    'EU_STRING',
    'ITEM_CODE',
    'ITEM_CODE_CPC', 
    'ITEM', 
    'ELEMENT_CODE',
    'ELEMENT', 
    'MONTHS_CODE', 
    'MONTHS', 
    'UNIT', 
    'WHEAT_PPI',
    'GDP',
]]
# MAKE INTERACTIVE
idf_gdp_ppi = df_gdp_ppi_reordered.interactive()

In [32]:
# GDP | WHEAT PPI PER YEAR BY COUNTRY
# 
gdp_ppi_pipeline = (
    idf_gdp_ppi[
        (idf_gdp_ppi.AREA == country_selector )
    ]
    .sort_values(by='YEAR',)
    .reset_index(drop=True)
)
# 

In [33]:
gdp_ppi_pipeline
# pn.Row(
#     pn.pane.Str(background='#f0f0f0', height=100, sizing_mode='stretch_width'),
#     width_policy='max', height=200,   
# )

In [34]:
# CREATE TABLE TO DISPLAY
# https://panel.holoviz.org/reference/widgets/Tabulator.html
# FORMATTING: https://panel.holoviz.org/reference/widgets/Tabulator.html#formatters
# =====
# SOMETIMES THE PIPELINE NEEDS TO RECREATED - NOT SURE WHY
# =====
# columns_to_show = ['AREA', 'LOOKUP_ISO', 'YEAR', 'GDP', 'WHEAT_PPI']
# 
gdp_ppi_table = gdp_ppi_pipeline.pipe(pn.widgets.Tabulator, 
                                      sizing_mode='stretch_width', 
                                      theme='bulma',
                                      hidden_columns=[
                                        'LOCATION', 
                                        'INDICATOR', 
                                        'SUBJECT', 
                                        'MEASURE', 
                                        'FREQUENCY', 
                                        'FLAG_CODES',
                                        'LOOKUP_ISO', 
                                        'AREA_CODE', 
                                        'AREA_CODE_M49', 
                                        'EU_MEMBER', 
                                        'ITEM_CODE', 
                                        'ITEM_CODE_CPC', 
                                        'ITEM', 
                                        'ELEMENT_CODE',
                                        'ELEMENT', 
                                        'MONTHS_CODE', 
                                        'MONTHS', 
                                        'UNIT',
                                        'EU_STRING'
                                      ],
                                      formatters={
                                          'YEAR': StringFormatter(), 
                                          'GDP': NumberFormatter(format='$0,0.00')
                                      },
                                      show_index=False) 

In [35]:
# DISPLAY TABLE
gdp_ppi_table

In [36]:
ppi_type_plot = gdp_ppi_pipeline.hvplot(kind='bar', 
                                             x='YEAR', 
                                             y="WHEAT_PPI", 
                                             title='Wheat PPI Breakdown')

In [37]:
ppi_type_plot

In [38]:
gdp_type_plot = gdp_ppi_pipeline.hvplot(kind='scatter', 
                                             x='WHEAT_PPI', 
                                             y="GDP", 
                                             title='GDP Breakdown')

In [39]:
gdp_type_plot

In [40]:
#  INLINE DASHBOARD

In [41]:
pn.Row(pn.Column(gdp_type_plot, ppi_type_plot,))

Row(sizing_mode='stretch_width')
    [0] Column(sizing_mode='stretch_width')
        [0] Interactive(Interactive, sizing_mode='stretch_width')
        [1] Interactive(Interactive, sizing_mode='stretch_width')

In [42]:
# STARTING DASHBOARD PROPERLY
# BASED ON: https://github.com/thu-vu92/python-dashboard-panel/blob/main/Interactive_dashboard.ipynb
# 

# DASHBOARD

In [43]:
# DEFINE YEAR SLIDER
# https://panel.holoviz.org/reference/widgets/IntSlider.html
# Define Panel widgets
year_slider = pn.widgets.IntSlider(name='Year', start=2000, end=2021, step=1, value=2001, bar_color="#348796")
year_slider

IntSlider(bar_color='#348796', end=2021, name='Year', sizing_mode='stretch_width', start=2000, value=2001)

In [44]:
# EU ONLY BUTTON
# https://panel.holoviz.org/reference/widgets/RadioButtonGroup.html
# Radio buttons for CO2 measures
breakdown_button_group = pn.widgets.RadioButtonGroup(
    name='GDP v Wheat PPI', 
    options=["GDP", 'WHEAT_PPI'],
    button_type='primary'
)

In [45]:
breakdown_button_group

RadioButtonGroup(button_type='primary', name='GDP v Wheat PPI', options=['GDP', 'WHEAT_PPI'], sizing_mode='stretch_width', value='GDP')

In [46]:
# eu_button_group.value

In [47]:
# CREATE TWO PIPELINES

In [48]:
# PIPELINE 1

In [49]:
# idf_gdp_ppi

In [50]:
# MATTHEW BELOW CELL PLEASE

In [51]:
gdp_ppi_year_pipeline = (
    idf_gdp_ppi[
        (idf_gdp_ppi.YEAR == year_slider)
        # (idf.AREA.isin(eu_button_group))
    ]
    .groupby(['AREA', 'YEAR'])[breakdown_button_group].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='YEAR')  
    .reset_index(drop=True)
)

In [52]:
gdp_ppi_year_pipeline

In [54]:
# UNRESTRAINED PIPELINE 1
idf_gdp_ppi_mask = (idf_gdp_ppi.YEAR == year_slider)

gdp_ppi_year_unrestrained_pipeline = (
    idf_gdp_ppi[
        idf_gdp_ppi_mask
    ]
#     .groupby(['AREA', 'YEAR'])[breakdown_button_group].mean()
#     .to_frame()
    .reset_index()
    .sort_values(by='YEAR')  
    .reset_index(drop=True)
)

In [55]:
gdp_ppi_unrestrained_table = gdp_ppi_year_unrestrained_pipeline.pipe(pn.widgets.Tabulator, 
                                      sizing_mode='stretch_width', 
                                      theme='bulma',
                                      hidden_columns=[
                                        'LOCATION', 
                                        'INDICATOR', 
                                        'SUBJECT', 
                                        'MEASURE', 
                                        'FREQUENCY', 
                                        'FLAG_CODES',
                                        'LOOKUP_ISO', 
                                        'AREA_CODE', 
                                        'AREA_CODE_M49', 
                                        'EU_MEMBER', 
                                        'ITEM_CODE', 
                                        'ITEM_CODE_CPC', 
                                        'ITEM', 
                                        'ELEMENT_CODE',
                                        'ELEMENT', 
                                        'MONTHS_CODE', 
                                        'MONTHS', 
                                        'UNIT',
                                        'EU_STRING',
                                        'index'
                                      ],
                                      formatters={
                                          'YEAR': StringFormatter(), 
                                          'GDP': NumberFormatter(format='$0,0.00')
                                      },
                                      show_index=False) 

In [56]:
gdp_ppi_unrestrained_table

In [57]:
# gdp_ppi_year_unrestrained_pipeline

In [58]:
# 
gdp_ppi_year_unrestrained_plot = gdp_ppi_year_unrestrained_pipeline.hvplot(x='AREA', by='YEAR', y=breakdown_button_group, kind='bar', line_width=2, title="GDP or WHEAT PPI v Year")
gdp_ppi_year_unrestrained_plot
# 
# 

In [59]:
# PIPELINE 2

In [60]:
ppi_vs_gdp_scatterplot_pipeline = (
    idf_gdp_ppi[
        (idf_gdp_ppi.YEAR == year_slider)
    ]
#     .groupby(['AREA', 'YEAR'])[breakdown_button_group].mean()
#     .to_frame()
    .reset_index()
    .sort_values(by='YEAR')  
    .reset_index(drop=True)
)

In [61]:
ppi_vs_gdp_scatterplot = ppi_vs_gdp_scatterplot_pipeline.hvplot(x='GDP', 
                                                                y='WHEAT_PPI', 
                                                                by='AREA', 
                                                                size=80, 
                                                                kind="scatter", 
                                                                alpha=0.7,
                                                                legend=False, 
                                                                height=500, 
                                                                width=500)
ppi_vs_gdp_scatterplot

In [62]:
country_list = df_gdp_ppi.AREA.unique().tolist()
# print(country_list)
multi_choice_country = pn.widgets.MultiChoice(name='Select Countries', value=['Ireland'],
    options=country_list)
multi_choice_country.value

['Ireland']

In [63]:
multi_choice_country

MultiChoice(name='Select Countries', options=['Austria', 'Belgium', ...], sizing_mode='stretch_width', value=['Ireland'])

In [72]:
ppi_vs_gdp_scatterplot_year_pipeline = (
    idf_gdp_ppi[
        (idf_gdp_ppi.AREA.isin(multi_choice_country))
    ]
    .reset_index()
    .sort_values(by='YEAR')  
    .reset_index(drop=True)
)

In [73]:
# RESTRICT TABLE BY YEAR TOO
ppi_vs_gdp_table_country_year_pipeline = (
    idf_gdp_ppi[
        (idf_gdp_ppi.AREA.isin(multi_choice_country)) &
        (idf_gdp_ppi.YEAR == year_slider)
    ]
    .reset_index()
    .sort_values(by='YEAR')  
    .reset_index(drop=True)
)

In [85]:
# TABLE LIMITED TO YEAR
gdp_ppi_year_table = ppi_vs_gdp_table_country_year_pipeline.pipe(pn.widgets.Tabulator, 
                                      sizing_mode='stretch_width', 
                                      theme='bulma',
                                                                 
                                      hidden_columns=[
                                        'LOCATION', 
                                        'INDICATOR', 
                                        'SUBJECT', 
                                        'MEASURE', 
                                        'FREQUENCY', 
                                        'FLAG_CODES',
                                        'LOOKUP_ISO', 
                                        'AREA_CODE', 
                                        'AREA_CODE_M49', 
                                        'EU_MEMBER', 
                                        'ITEM_CODE', 
                                        'ITEM_CODE_CPC', 
                                        'ITEM', 
                                        'ELEMENT_CODE',
                                        'ELEMENT', 
                                        'MONTHS_CODE', 
                                        'MONTHS', 
                                        'UNIT',
                                        'EU_STRING',
                                        'index'
                                      ],
                                      formatters={
                                          'YEAR': StringFormatter(), 
                                          'GDP': NumberFormatter(format='$0,0.00')
                                      },
                                      show_index=False)

In [86]:
gdp_ppi_full_table = ppi_vs_gdp_scatterplot_year_pipeline.pipe(pn.widgets.Tabulator, 
                                      sizing_mode='stretch_width', 
                                      theme='bulma',
                                                                 
                                      hidden_columns=[
                                        'LOCATION', 
                                        'INDICATOR', 
                                        'SUBJECT', 
                                        'MEASURE', 
                                        'FREQUENCY', 
                                        'FLAG_CODES',
                                        'LOOKUP_ISO', 
                                        'AREA_CODE', 
                                        'AREA_CODE_M49', 
                                        'EU_MEMBER', 
                                        'ITEM_CODE', 
                                        'ITEM_CODE_CPC', 
                                        'ITEM', 
                                        'ELEMENT_CODE',
                                        'ELEMENT', 
                                        'MONTHS_CODE', 
                                        'MONTHS', 
                                        'UNIT',
                                        'EU_STRING',
                                        'index'
                                      ],
                                      formatters={
                                          'YEAR': StringFormatter(), 
                                          'GDP': NumberFormatter(format='$0,0.00')
                                      },
                                      show_index=False)

In [87]:
gdp_ppi_year_table

In [66]:
# gdp_ppi_year_table

In [78]:
# SCATTER
ppi_vs_gdp_scatterplot_noyear = ppi_vs_gdp_scatterplot_year_pipeline.hvplot(x='GDP', 
                                                                y='WHEAT_PPI', 
                                                                by='AREA', 
                                                                size=80, 
                                                                kind="scatter",
                                                                title="Wheat v GDP",
                                                                alpha=0.7,
                                                                legend=True, 
                                                                height=600, 
                                                                width=500)
ppi_vs_gdp_scatterplot_noyear

In [115]:
# tabs = pn.Tabs(('Limited to Year', gdp_ppi_year_table), ('Full Year',gdp_ppi_full_table))

In [106]:
# tabs

In [113]:

template = pn.template.FastListTemplate(
    title='Wheat PPI v GDP Dashboard', 
    sidebar=[pn.pane.Markdown("## Country Breakdown"), 
             pn.pane.Markdown("### Settings for Scatter"),   
             multi_choice_country,  
             pn.pane.Markdown("### Settings for Table"),
             year_slider
            ],
    main=[
        pn.Row(
            pn.Column(
                ppi_vs_gdp_scatterplot_noyear.panel(width=600), 
#                 gdp_ppi_year_table.panel(width=600),
                pn.Tabs(
                    ('Year Limited', pn.Row( gdp_ppi_year_table.panel(width=600), margin=(10,10)) ), 
                    ('All Years', pn.Row( gdp_ppi_full_table.panel(width=600), margin=(10,10)) ) ),
                margin=(0,0)
            ),
        )],
    sidebar_footer="Megan McHugh",
    theme_toggle=False,
    accent="#00733B",
#     accent_base_color="#00733B",
    corner_radius=4,
)

In [114]:
template.show()

Launching server at http://localhost:49698
